## Tokenized news를 TFIDF로 벡터화하고 DBSCAN으로 군집화

In [83]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import numpy as np
import pandas as pd
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import scale
import mglearn
from sklearn.metrics import silhouette_samples, silhouette_score

In [84]:
df_tmp = pd.read_csv('../data/tokenized_news/tokenized_all')

In [85]:
df_tmp

,Unnamed: 0,title,text,url,media,date,text_V,title_V
0,0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...","일과, 경계, 노동"
1,1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"일자리, 일거리, 노동, 미래, 현실, 변화, 사회, 준비, 짐작, 노동, 미래, ...","일자리, 일거리, 정치, 노동"
2,2,이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\nㆍAI 학습데이터 가공 기업 ‘크라우드웍스’ 교육장 참관...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"일자리, 일거리, 학습, 데이터, 가공, 기업, 우드, 웍스, 교육장, 참관, 소화...","기계, 정확도, 인간, 부품, 육성, 공장, 노동"
3,3,"지혜씨, 지훈씨를 부탁해! [인포그래픽]",새로 태어나는 아이들은 줄어들고 어르신들은 계속 나이를 먹어 갑니다. 2020년 새...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"어르신, 계속, 나이, 새해, 크게, 정부, 정책, 발표, 혜택, 저출산, 고령, ...","지혜, 지훈, 부탁, 인포그래픽"
4,4,"[20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”",새천년의 시작과 함께 세상에 나온 2000년대생들이 어느덧 사회 변화를 이끌 20대...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-01-01,"새천년, 시작, 세상, 년대, 사회, 변화, 대가, 밀레니엄, 베이비, 세대, 분류...","리포트, 함부로, 판단, 간다"
...,...,...,...,...,...,...,...,...
29036,29036,"""2.5단계로 무급휴가 요구""… 코로나 재유행에 살아난 '갑질'",직장인 A씨는 수도권 사회적 거리두기 2.5단계를 시행하는 30일부터 다음달 6일...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31,"직장인, 수도권, 사회, 거리, 두기, 단계, 시행, 일간, 무급, 휴가, 사용, ...","단계, 무급, 휴가, 요구, 코로나, 유행"
29037,29037,모호한 조치ㆍ 깜깜이 환자 비율 40%육박... 거리두기 2.5단계 효과 있을까,"30일 0시부터 서울 전역에 강화된 사회적 거리두기(2.5단계)가 시행됐지만, 전문...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31,"서울, 전역, 강화, 사회, 거리, 두기, 단계, 시행, 전문가, 방역, 효과, 의...","조치, 환자, 비율, 육박, 거리, 두기, 단계, 효과"
29038,29038,내일부터 서울 시내버스 밤 9시 이후 20% 감축… 주말 수준으로,수도권의 ‘사회적 거리두기’ 2.5단계 시행 기간에 맞춰 서울 시내버스의 야간 운행...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31,"수도권, 사회, 거리, 두기, 단계, 시행, 기간, 서울, 시내버스, 야간, 운행,...","내일, 서울, 시내버스, 감축, 주말, 수준"
29039,29039,집단휴업 두고 전공의 vs 정부 '강대강 대치'... 의료공백 장기화 불가피,21일부터 10일째 무기한 집단휴진을 이어가고 있는 전공의(인턴ㆍ레지던트)들과 정부...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31,"무기한, 집단, 어가, 전공, 인턴, 레지던트, 정부, 갈등, 벼랑, 의협, 의회,...","집단, 휴업, 전공, 정부, 대강, 대치, 의료, 공백, 장기, 불가피"


In [86]:
tfidf_vect = TfidfVectorizer(max_df=0.95)

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(df_tmp['title_V'].apply(lambda x: np.str_(x)))

In [87]:
dbscan = DBSCAN(eps=0.2, min_samples=3, metric='euclidean')
dbscan_labels = dbscan.fit_predict(feature_vect)

In [88]:
df_tmp['dbscan_cluster'] = dbscan_labels

In [89]:
df_tmp[df_tmp['dbscan_cluster']==0].sort_values(by='date')

,Unnamed: 0,title,text,url,media,date,text_V,title_V,dbscan_cluster
68,68,1월 1일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■ 한겨레 그림판 바로가기\n\...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-01-01,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
188,188,1월 2일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■ 한겨레 그림판 바로가기\n\...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-01-02,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
318,318,1월 3일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■ 한겨레 그림판 바로가기\n\...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-01-03,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
529,529,1월 6일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n한겨레 그림판 바로가기\n\n\...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-01-06,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
673,673,1월 7일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■ 한겨레 그림판 바로가기\n\...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-01-07,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
...,...,...,...,...,...,...,...,...,...
24093,24093,7월 21일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■한겨레 그림판 바로가기\n\n...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-07-21,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
24250,24250,7월 22일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■한겨레 그림판 바로가기\n\n...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-07-22,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
24962,24962,7월 28일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■한겨레 그림판 바로가기\n\n...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-07-28,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0
26671,26671,8월 11일 한겨레 그림판,권범철 기자 kartoon@hani.co.kr\n\n■한겨레 그림판 바로가기\n\n...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-08-11,"권범철, 기자, 한겨레, 그림판, 바로가기, 한겨레신문, 무단, 배포, 금지","한겨레, 그림판",0


In [90]:
df_tmp[df_tmp['dbscan_cluster']==1].sort_values(by='date')

,Unnamed: 0,title,text,url,media,date,text_V,title_V,dbscan_cluster
94,94,2020년 1월 2일 [김용민의 그림마당],"©경향신문(www.khan.co.kr), 무단전재 및 재배포 금지",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-02,"무단, 배포, 금지","김용민, 그림, 마당",1
214,214,2020년 1월 3일 [김용민의 그림마당],"©경향신문(www.khan.co.kr), 무단전재 및 재배포 금지",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-03,"무단, 배포, 금지","김용민, 그림, 마당",1
426,426,2020년 1월 6일 [김용민의 그림마당],"©경향신문(www.khan.co.kr), 무단전재 및 재배포 금지",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-06,"무단, 배포, 금지","김용민, 그림, 마당",1
560,560,2020년 1월 7일 [김용민의 그림마당],"©경향신문(www.khan.co.kr), 무단전재 및 재배포 금지",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-07,"무단, 배포, 금지","김용민, 그림, 마당",1
705,705,2020년 1월 8일 [김용민의 그림마당],"©경향신문(www.khan.co.kr), 무단전재 및 재배포 금지",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-08,"무단, 배포, 금지","김용민, 그림, 마당",1
...,...,...,...,...,...,...,...,...,...
28204,28204,2020년 8월 25일 [김용민의 그림마당],"[경향신문] \n\n\n\n\n\n\n\n©경향신문(www.khan.co.kr), ...",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-08-25,"무단, 배포, 금지","김용민, 그림, 마당",1
28355,28355,2020년 8월 26일 [김용민의 그림마당],"[경향신문] \n\n\n\n\n\n\n\n©경향신문(www.khan.co.kr), ...",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-08-26,"무단, 배포, 금지","김용민, 그림, 마당",1
28507,28507,2020년 8월 27일 [김용민의 그림마당],"[경향신문] \n\n\n\n\n\n\n\n©경향신문(www.khan.co.kr), ...",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-08-27,"무단, 배포, 금지","김용민, 그림, 마당",1
28658,28658,김용민의 그림마당 쉽니다,[경향신문] \n※ ‘그림마당’ 쉽니다.\n\n\n\n\n\n©경향신문(www.kh...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-08-28,"그림, 마당, 무단, 배포, 금지","김용민, 그림, 마당",1


In [91]:
df_tmp[df_tmp['dbscan_cluster']==-1].sort_values(by='date')

,Unnamed: 0,title,text,url,media,date,text_V,title_V,dbscan_cluster
0,0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...","일과, 경계, 노동",-1
60,60,가짜가 진짜보다 6배 빨리 퍼진다… 인류 파괴하는 '거짓 정보',[조선일보 100년 / 진실의 수호자들] [1] 저널리즘의 도전: 민주주의를 위협하...,https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-01,"진실, 호자, 저널리즘, 도전, 민주주의, 위협, 가짜, 전쟁, 대선, 힐러리, 아...","가짜, 진짜, 인류, 파괴, 거짓, 정보",-1
59,59,"""진짜뉴스는 비싸다, 때론 목숨까지 건다""",[조선일보 100년 / 진실의 수호자들] [1] 저널리즘의 도전: 민주주의를 위협하...,https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-01,"진실, 호자, 저널리즘, 도전, 민주주의, 위협, 가짜, 전쟁, 거짓, 정보, 손가...","진짜, 뉴스, 때론, 목숨",-1
58,58,"""아들 美대학 시험, 조국부부가 대신 풀어줘 A학점""","檢 ""아들이 문제 사진찍어 보내면 부부가 답안 적어 다시 美 보내… 美대학 장학증명...",https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-01,"아들, 문제, 사진, 부부, 답안, 다시, 대학, 장학, 증명서, 위조, 국회, 제...","아들, 대학, 시험, 조국, 부부, 대신, 학점",-1
57,57,"現 국회서 공수처장 추천위는 범여6·野1 구도 ""살아있는 권력 수사, 유재수가 끝이...","野추천 2명 중 1명 군소정당 몫\n정부·여당, 21대 국회 시작전에 '4+1' 또...",https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-01,"추천, 군소, 정당, 정부, 여당, 국회, 작전, 활용, 추천, 임명, 고위, 공직...","국회, 추천, 범여, 구도, 권력, 수사, 유재수",-1
...,...,...,...,...,...,...,...,...,...
28942,28942,대부분 업소·손님 ‘자발적 통금’… 일부는 ‘방역 미꾸라지’,■ ‘2.5단계 방역지침’ 시행\n\n강남 업소 대부분 ‘시간’ 준수\n\n21시 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"단계, 방역, 지침, 시행, 강남, 대부분, 준수, 음식, 포장, 대치동, 소형, ...","대부분, 손님, 자발, 통금, 일부, 방역, 미꾸라지",-1
28944,28944,학원·독서실 모두 문닫아… 공부할 곳 잃은 학생들,“집중해 공부할곳 없다” 하소연\n\n공부방·과외 등에 학생 몰려\n\n신종 코로나...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"집중, 공부, 하소연, 부방, 과외, 학생, 신종, 코로나바이러스, 감염증, 코로나...","학원, 독서실, 공부, 학생",-1
28945,28945,"민주당, 2차 재난지원금 지급·4차 추경 논의 가속도",이르면 내일 고위 당정청 개최\n\n추석 민생대책으로 내놓을수도\n\n더불어민주당이...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"내일, 고위, 정청, 개최, 추석, 민생, 대책, 민주당, 재난, 원금, 지급, 추...","민주당, 재난, 원금, 지급, 추경, 논의, 가속도",-1
28947,28947,진보성향 시민단체까지 의료진 고발 가세,"“파업으로 응급환자 사망 초래” \n\n적폐청산연대, 살인혐의 주장\n\n친여·진...",https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"파업, 응급환자, 사망, 초래, 적폐, 청산, 연대, 살인, 혐의, 친, 진보, 성...","진보, 성향, 시민단체, 의료, 고발, 가세",-1


In [92]:
df_tmp.sort_values(by='dbscan_cluster')

,Unnamed: 0,title,text,url,media,date,text_V,title_V,dbscan_cluster
0,0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...","일과, 경계, 노동",-1
19297,19297,서울·수도권 집값 다시 들썩… 정부 “언제든 필요 조치” 경고,"洪 부총리, 추가 대책 시사/ 서울 아파트값 10주 만에 상승/ 안산·인천·군포 움...",https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-06-12,"부총리, 추가, 대책, 시사, 서울, 아파트, 상승, 안산, 인천, 군포, 움직임,...","서울, 수도권, 집값, 다시, 정부, 필요, 조치, 경고",-1
19296,19296,美 올 경제성장률 -6.5%… 2022년까지 제로 금리,연준 “코로나 충격 회복에 수년/ 연말 회복세… 불확실성 아직 커”/ 나스닥 출범 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-06-12,"준, 코로나, 충격, 회복, 수년, 연말, 회복, 불확실, 나스닥, 출범, 미국, ...","경제성장률, 제로, 금리",-1
19295,19295,‘이재용 사건’ 수사심의위 연다,시민위원들 과반수 찬성 가결/ 기소 여부 외부전문가들이 검토/ 檢 “결정 존중”… ...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-06-12,"시민, 위원, 과반수, 찬성, 가결, 기소, 외부, 전문가, 검토, 결정, 존중, ...","이재용, 사건, 수사, 심의",-1
19294,19294,"징역 18년 확정…‘국정농단’ 최서원 측 ""억울""","대법, 벌금 200억 등 원심대로/ 안종범 징역 4년·벌금 6000만원\n\n박근혜...",https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-06-12,"대법, 벌금, 원심, 안종범, 징역, 벌금, 만원, 박근혜정부, 서원, 개명, 실,...","징역, 확정, 국정, 농단, 서원, 억울",-1
...,...,...,...,...,...,...,...,...,...
25760,25760,윤석열 “민주주의 허울 쓴 독재 배격해야”,신임검사 신고식서 작심 발언\n“권력형 비리에 당당하게 맞서야” \n정부·여권 거...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-08-04,"신임, 검사, 신고식, 작심, 발언, 권력, 비리, 정부, 여권, 압박, 소신, 추...","윤석열, 민주주의, 허울, 독재, 배격",24
25717,25717,윤석열 “민주주의 허울 쓴 독재 배격”,윤석열 검찰총장(사진)이 신임 검사들을 상대로 공정한 법집행의 필요성을 강조했다. ...,https://news.naver.com/main/read.nhn?mode=LPOD...,동아일보,2020-08-04,"윤석열, 검찰총장, 사진, 신임, 검사, 상대로, 법, 집행, 필요성, 강조, 총장...","윤석열, 민주주의, 허울, 독재, 배격",24
26297,26297,부동산 정책 다루는 고위공직자 10명 중 4명이 다주택자,부동산ㆍ금융정책을 다루는 주요 부처와 산하기관의 고위공직자 10명 중 4명이 주택 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-07,"부동산, 금융정책, 주요, 부처, 기관, 고위, 공직자, 주택, 보유, 주택, 조사...","부동산, 정책, 고위, 공직자, 주택",25
26280,26280,부동산 정책 고위공직자 36%가 다주택,"부동산 정책 관련 주요 부처 고위공직자의 부동산 재산이 국민 평균보다 4배나 많고,...",https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-08-07,"부동산, 정책, 주요, 부처, 고위, 공직자, 부동산, 재산, 국민, 평균, 주택,...","부동산, 정책, 고위, 공직자, 주택",25


In [93]:
print("클러스터 레이블:\n{}".format(dbscan_labels))

클러스터 레이블:
[-1 -1 -1 ... -1 -1 -1]


In [94]:
score_samples = silhouette_samples(feature_vect, df_tmp['dbscan_cluster'])
print('silhouette_samples return 값의 shape' , score_samples.shape)

silhouette_samples return 값의 shape (29041,)


In [95]:
# 모든 데이터의 평균 실루엣 계수를 구함
average_score = silhouette_score(feature_vect, df_tmp['dbscan_cluster'])

In [96]:
print('뉴스 타이틀 Silhouette Analysis Score:{0:.3f}'.format(average_score))
df_tmp['dbscan_cluster']

뉴스 타이틀 Silhouette Analysis Score:-0.010


0       -1
1       -1
2       -1
3       -1
4       -1
        ..
29036   -1
29037   -1
29038   -1
29039   -1
29040   -1
Name: dbscan_cluster, Length: 29041, dtype: int64

In [97]:
df_tmp['title'][df_tmp['dbscan_cluster']==0]

68        1월 1일 한겨레 그림판
188       1월 2일 한겨레 그림판
318       1월 3일 한겨레 그림판
529       1월 6일 한겨레 그림판
673       1월 7일 한겨레 그림판
              ...      
24093    7월 21일 한겨레 그림판
24250    7월 22일 한겨레 그림판
24962    7월 28일 한겨레 그림판
26671    8월 11일 한겨레 그림판
26824    8월 12일 한겨레 그림판
Name: title, Length: 92, dtype: object

In [98]:
df_tmp['title'][df_tmp['dbscan_cluster']==1]

94        2020년 1월 2일 [김용민의 그림마당]
214       2020년 1월 3일 [김용민의 그림마당]
426       2020년 1월 6일 [김용민의 그림마당]
560       2020년 1월 7일 [김용민의 그림마당]
705       2020년 1월 8일 [김용민의 그림마당]
                   ...           
28204    2020년 8월 25일 [김용민의 그림마당]
28355    2020년 8월 26일 [김용민의 그림마당]
28507    2020년 8월 27일 [김용민의 그림마당]
28658               김용민의 그림마당 쉽니다
28900    2020년 8월 31일 [김용민의 그림마당]
Name: title, Length: 162, dtype: object

In [99]:
df_tmp['title'][df_tmp['dbscan_cluster']==-1]

0                           무너지는 일과 삶의 경계···노동이 녹아내린다
1                일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동]
2          이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동]
3                              지혜씨, 지훈씨를 부탁해! [인포그래픽]
4                [20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”
                             ...                     
29036             "2.5단계로 무급휴가 요구"… 코로나 재유행에 살아난 '갑질'
29037    모호한 조치ㆍ 깜깜이 환자 비율 40%육박... 거리두기 2.5단계 효과 있을까
29038            내일부터 서울 시내버스 밤 9시 이후 20% 감축… 주말 수준으로
29039      집단휴업 두고 전공의 vs 정부 '강대강 대치'... 의료공백 장기화 불가피
29040              "제자 한명이라도 피해 입으면 집단휴진" 의대 교수까지 화났다
Name: title, Length: 28505, dtype: object